In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import os

a)

In [2]:

def parseador_pitstops(url: str):
    try:
        xml_data = requests.get(url).content
        root = ET.fromstring(xml_data)
        namespace = {'ns': 'http://ergast.com/mrd/1.5'}
        
        # Obtener el tipo de carrera de la URL
        race_url = root.find('.//ns:Race', namespace).get('url')
        race_type = race_url.split("/")[-1]  # Extraer la parte final de la URL
        
        pitstops_data = []
        pitstops_list = root.find('.//ns:PitStopsList', namespace)
        
        for pitstop in pitstops_list.findall('ns:PitStop', namespace):
            driver_id = pitstop.get('driverId')
            stop_number = pitstop.get('stop')
            lap = pitstop.get('lap')
            time = pitstop.get('time')
            duration = pitstop.get('duration')
            
            # Agregar el tipo de carrera a los datos de pitstop
            pitstops_data.append({'DriverID': driver_id, 'Stop': stop_number, 'Lap': lap, 'Time': time, 'Duration': duration, 'RaceType': race_type})
        
        pitstops_df = pd.DataFrame(pitstops_data)
        return pitstops_df
    except AttributeError:
        return pd.DataFrame()

In [3]:
print(pd.DataFrame().empty)

True


In [24]:
# esta parte tarda un poco (3 mins)
dfs_years = {'2012': [], '2013': [], '2014': [], '2015': [], '2016': [], '2017': [], '2018': [], '2019': [], '2020': [], '2021': [], '2022': [], '2023': []}
for year in dfs_years.keys():
    print(year)
    ronda = 1
    while ronda != 100:
        offset = 0
        while True:
            final = f'?limit=30&offset={offset}'
            url = f'https://ergast.com/api/f1/{year}/{ronda}/pitstops{final}'
            pitstops_df = parseador_pitstops(url)
            if not pitstops_df.empty:
                pitstops_df['year'] = [year for i in range(len(pitstops_df))]
                pitstops_df['round'] = [ronda for i in range(len(pitstops_df))]
                dfs_years[year].append(pitstops_df)
                offset += 30
            else:
                break
        ronda += 1
all_pitstops = pd.concat([df for dfs_list in dfs_years.values() for df in dfs_list], ignore_index=True)


2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [44]:
print(all_pitstops[all_pitstops['RaceType'] == '2021_Azerbaijan_Grand_Prix'].tail())

             DriverID Stop Lap      Time  Duration  \
7183           bottas    3  48  17:37:13       NaN   
7184          russell    4  48  17:37:18       NaN   
7185          mazepin    5  47  17:37:20       NaN   
7186  mick_schumacher    5  48  17:37:47       NaN   
7187          mazepin    6  48  18:09:58       NaN   

                        RaceType  year  round  
7183  2021_Azerbaijan_Grand_Prix  2021      6  
7184  2021_Azerbaijan_Grand_Prix  2021      6  
7185  2021_Azerbaijan_Grand_Prix  2021      6  
7186  2021_Azerbaijan_Grand_Prix  2021      6  
7187  2021_Azerbaijan_Grand_Prix  2021      6  


In [26]:
print(all_pitstops.groupby('year')['RaceType'].nunique())

year
2012    20
2013    19
2014    19
2015    19
2016    21
2017    20
2018    21
2019    21
2020    17
2021    21
2022    22
2023    22
Name: RaceType, dtype: int64


b)

In [27]:

driver_ids = list(set(all_pitstops['DriverID']))
print(driver_ids)
print(len(driver_ids))
print(driver_ids.index('max_verstappen'))
    


['stevens', 'bruno_senna', 'rosa', 'chilton', 'kubica', 'vandoorne', 'raikkonen', 'kovalainen', 'jules_bianchi', 'haryanto', 'zhou', 'wehrlein', 'ambrosio', 'alonso', 'ricciardo', 'michael_schumacher', 'kevin_magnussen', 'aitken', 'sutil', 'pic', 'glock', 'norris', 'rossi', 'hamilton', 'nasr', 'resta', 'maldonado', 'piastri', 'hulkenberg', 'ericsson', 'vergne', 'bottas', 'ocon', 'webber', 'pietro_fittipaldi', 'vettel', 'grosjean', 'button', 'sirotkin', 'albon', 'sargeant', 'de_vries', 'sainz', 'kobayashi', 'tsunoda', 'garde', 'gasly', 'perez', 'mick_schumacher', 'giovinazzi', 'merhi', 'jolyon_palmer', 'leclerc', 'massa', 'rosberg', 'mazepin', 'brendon_hartley', 'russell', 'latifi', 'petrov', 'karthikeyan', 'stroll', 'gutierrez', 'lawson', 'max_verstappen', 'kvyat']
66
64


In [29]:
def parseador_drivers(url:str):
    try: 
        xml_data = requests.get(url).content
        root = ET.fromstring(xml_data)
        namespace = {'ns': 'http://ergast.com/mrd/1.5'}
        driver_id = root.find('.//ns:Driver', namespace).get('driverId')
        code = root.find('.//ns:Driver', namespace).get('code')
        # url = root.find('.//ns:Driver', namespace).get('url')
        permanent_number = root.find('.//ns:PermanentNumber', namespace).text
        given_name = root.find('.//ns:GivenName', namespace).text
        family_name = root.find('.//ns:FamilyName', namespace).text
        date_of_birth = root.find('.//ns:DateOfBirth', namespace).text
        nationality = root.find('.//ns:Nationality', namespace).text
        data = {
            'DriverID': [driver_id],
            'Code': [code],
            'PermanentNumber': [permanent_number],
            'GivenName': [given_name],
            'FamilyName': [family_name],
            'DateOfBirth': [date_of_birth],
            'Nationality': [nationality]
        }
        '''
        data = {
            'DriverID': [driver_id],
            'Code': [code],
            'URL': [url],
            'PermanentNumber': [permanent_number],
            'GivenName': [given_name],
            'FamilyName': [family_name],
            'DateOfBirth': [date_of_birth],
            'Nationality': [nationality]
        }
        '''
        df = pd.DataFrame(data)
        return df
    except AttributeError:
        return driver_id




In [30]:
drivers_dfs = []
pilotos_retirados = [] # Los pilotos retirados no tienen numero y salta error 
for driverid in driver_ids:
    url = f'https://ergast.com/api/f1/drivers/{driverid}'
    resultado = parseador_drivers(url)
    if type(resultado) ==  str:
        pilotos_retirados.append(resultado)
    else:
        drivers_dfs.append(resultado)

drivers_df = pd.concat(drivers_dfs)
    


In [10]:
print(pilotos_retirados)

['bruno_senna', 'rosa', 'kovalainen', 'ambrosio', 'michael_schumacher', 'pic', 'glock', 'webber', 'garde', 'petrov', 'karthikeyan']


In [31]:
print(drivers_df.head())

    DriverID Code PermanentNumber GivenName FamilyName DateOfBirth Nationality
0    stevens  STE              28      Will    Stevens  1991-06-28     British
0    chilton  CHI               4       Max    Chilton  1991-04-21     British
0     kubica  KUB              88    Robert     Kubica  1984-12-07      Polish
0  vandoorne  VAN               2   Stoffel  Vandoorne  1992-03-26     Belgian
0  raikkonen  RAI               7      Kimi  Räikkönen  1979-10-17     Finnish


In [32]:
retirados_mapeo = pd.DataFrame()
retirados_mapeo['DriverID'] = pilotos_retirados
retirados_mapeo['PermanentNumber'] = [np.nan for i in range(len(pilotos_retirados))]
activos_mapeo = pd.DataFrame()
activos_mapeo['DriverID'] = drivers_df['DriverID']
activos_mapeo['PermanentNumber'] = drivers_df['PermanentNumber']
mapeo_df = pd.concat([activos_mapeo,retirados_mapeo])




In [33]:
c = 0
for id in driver_ids:
    print(mapeo_df[mapeo_df['DriverID'] == id]['PermanentNumber'].values)
    c+=1
print(c)


['28']
[nan]
[nan]
['4']
['88']
['2']
['7']
[nan]
['17']
['88']
['24']
['94']
[nan]
['14']
['3']
[nan]
['20']
['89']
['99']
[nan]
[nan]
['4']
['53']
['44']
['12']
['40']
['13']
['81']
['27']
['9']
['25']
['77']
['31']
[nan]
['51']
['5']
['8']
['22']
['35']
['23']
['2']
['21']
['55']
['10']
['22']
[nan]
['10']
['11']
['47']
['99']
['98']
['30']
['16']
['19']
['6']
['9']
['28']
['63']
['6']
[nan]
[nan]
['18']
['21']
['40']
['33']
['26']
66


In [34]:
mapeo_df.loc[mapeo_df['DriverID'] == 'max_verstappen', 'PermanentNumber'] = '33, 1' # Consideramos que verstappen cambió de número 

In [35]:
print(mapeo_df.loc[mapeo_df['DriverID'] == 'max_verstappen', 'PermanentNumber'].values[0])

33, 1


c)

In [36]:
all_pitstops['Duration'] = pd.to_numeric(all_pitstops['Duration'],errors='coerce')
print(all_pitstops['Duration'])

0       24.599
1       32.319
2       22.313
3       23.203
4       22.035
         ...  
9427    21.274
9428    21.667
9429    21.275
9430    21.919
9431    21.564
Name: Duration, Length: 9432, dtype: float64


In [37]:
carreras = all_pitstops['RaceType'].unique().tolist()
print(carreras)

['2012_Australian_Grand_Prix', '2012_Malaysian_Grand_Prix', '2012_Chinese_Grand_Prix', '2012_Bahrain_Grand_Prix', '2012_Spanish_Grand_Prix', '2012_Monaco_Grand_Prix', '2012_Canadian_Grand_Prix', '2012_European_Grand_Prix', '2012_British_Grand_Prix', '2012_German_Grand_Prix', '2012_Hungarian_Grand_Prix', '2012_Belgian_Grand_Prix', '2012_Italian_Grand_Prix', '2012_Singapore_Grand_Prix', '2012_Japanese_Grand_Prix', '2012_Korean_Grand_Prix', '2012_Indian_Grand_Prix', '2012_Abu_Dhabi_Grand_Prix', '2012_United_States_Grand_Prix', '2012_Brazilian_Grand_Prix', '2013_Australian_Grand_Prix', '2013_Malaysian_Grand_Prix', '2013_Chinese_Grand_Prix', '2013_Bahrain_Grand_Prix', '2013_Spanish_Grand_Prix', '2013_Monaco_Grand_Prix', '2013_Canadian_Grand_Prix', '2013_British_Grand_Prix', '2013_German_Grand_Prix', '2013_Hungarian_Grand_Prix', '2013_Belgian_Grand_Prix', '2013_Italian_Grand_Prix', '2013_Singapore_Grand_Prix', '2013_Korean_Grand_Prix', '2013_Japanese_Grand_Prix', '2013_Indian_Grand_Prix', '2

In [38]:
for carrera in carreras:
    if 'Dutch'  in carrera:
        print(carrera)

2021_Dutch_Grand_Prix
2022_Dutch_Grand_Prix
2023_Dutch_Grand_Prix


In [39]:
all_pitstops['Duration'] = pd.to_numeric(all_pitstops['Duration'],errors='coerce')
carreras = all_pitstops['RaceType'].unique().tolist()
races_dataframes = []

for carrera in carreras:
    df_dic = {"DriverID":[], "DriverNumber":[], "NPitstops":[], "MedianPitStopDuration":[]}
    pitstops_carrera = all_pitstops[all_pitstops['RaceType'] == carrera]
    driver_ids = pitstops_carrera['DriverID'].unique().tolist()
    
    for id in driver_ids:
        diver_number = mapeo_df.loc[mapeo_df['DriverID'] == id, 'PermanentNumber'].values[0]
        npitstops = len(pitstops_carrera[pitstops_carrera['DriverID'] == id])
        pitstop_durations = pitstops_carrera[pitstops_carrera['DriverID'] == id]['Duration']
        
        if not pitstop_durations.isnull().all():
            median_pitstop_duration = pitstop_durations.median()
            
        else:
            median_pitstop_duration = np.nan 
            
        df_dic['DriverID'].append(id)
        df_dic['DriverNumber'].append(diver_number)
        df_dic['NPitstops'].append(npitstops)
        df_dic['MedianPitStopDuration'].append(median_pitstop_duration)
        
    df = pd.DataFrame(df_dic)
    races_dataframes.append(df)

In [40]:
print(races_dataframes[0].info())
print(races_dataframes[68].info())
print(len(races_dataframes))
print(len(carreras))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DriverID               19 non-null     object 
 1   DriverNumber           13 non-null     object 
 2   NPitstops              19 non-null     int64  
 3   MedianPitStopDuration  19 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 736.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DriverID               18 non-null     object 
 1   DriverNumber           18 non-null     object 
 2   NPitstops              18 non-null     int64  
 3   MedianPitStopDuration  18 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 704.0+ bytes
None
242
242


d)

In [41]:
if not os.path.exists('Yearly grandprixs reports'):
    os.mkdir('Yearly grandprixs reports')
for i in range(len(races_dataframes)):
    carrera_name = carreras[i].replace('Grand_Prix','pitsops')
    if carrera_name == '70th_Anniversary_pitsops': # Esta carrera es del 2020
        if not os.path.exists('Yearly grandprixs reports/2020'):
            os.mkdir('Yearly grandprixs reports/2020')
        races_dataframes[i].to_csv(f'Yearly grandprixs reports/2020/{carrera_name}.csv',index=False)


    else:
        carrera_name = carrera_name.split('_')
        year = carrera_name[0]
        country_name = ''
        for i in range(1,len(carrera_name)-1):
            country_name += carrera_name[i] + '_'
        carrera_name = country_name + year + '_' + 'pitsops'

        if not os.path.exists(f'Yearly grandprixs reports/{year}'):
            os.mkdir(f'Yearly grandprixs reports/{year}')
        ruta = f'Yearly grandprixs reports/{year}/{carrera_name}.csv'
        races_dataframes[i].to_csv(ruta,index=False)
